In [1]:
!pip install torch transformers accelerate peft datasets bitsandbytes

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
                                              0.0/203.1 MB ? eta -:--:--
                                              0.0/203.1 MB ? eta -:--:--
                                            0.0/203.1 MB 165.2 kB/s eta 0:20:30
                                            0.0/203.1 MB 163.8 kB/s eta 0:20:40
                                            0.0/203.1 MB 163.8 kB/s eta 0:20:40
                                            0.0/203.1 MB 164.3 kB/s eta 0:20:37
                                            0.1/203.1 MB 204.8 kB/s eta 0:16:32
                                            0.1/203.1 MB 207.6 kB/s eta 0:16:19
                                            0.1/203.1 MB 207.6 kB/s eta 0:16:19
                                            0.1/203.1 MB 236.3 kB/s eta 0:14:20
                                            0.1/203.1 MB 267.6 kB/s eta 0:12:39
                                            0.1/203.1 MB 293.

In [4]:
from huggingface_hub import list_datasets
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
import bitsandbytes
#List out the data set available
class_dataset = list_datasets(full=True, filter="text-classification")
classification_dataset_names = [dataset.id for dataset in class_dataset]

print(f"There are {len(classification_dataset_names)} classification datasets available on the hub")
print(f"The first 10 are: {classification_dataset_names[:10]}")

There are 40 classification datasets available on the hub
The first 10 are: ['nace-ai/policy-alignment-verification-dataset', 'cvcio/toxic-el', 'ckandemir/bitcoin_tweets_sentiment_kaggle', 'danilocorsi/LLMs-Sentiment-Augmented-Bitcoin-Dataset', 'tanvircr7/learn_hf_food_not_food_image_captions', 'agentlans/high-quality-nli', 'ParsBench/PersianSyntheticEmotions', 'ronnieaban/alquran', 'MonoHime/ru_sentiment_dataset', 'cmotions/NL_restaurant_reviews']


In [5]:
tweet_sent = load_dataset("ckandemir/bitcoin_tweets_sentiment_kaggle")

c:\Users\hp\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\datasets--ckandemir--bitcoin_tweets_sentiment_kaggle. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating eval split: 100%|██████████| 9724/9724 [00:00<00:00, 1483051.97 examples/s]


In [9]:
tweet_sent

DatasetDict({
    train: Dataset({
        features: ['Date', 'text', 'Sentiment'],
        num_rows: 77791
    })
    test: Dataset({
        features: ['Date', 'text', 'Sentiment'],
        num_rows: 9724
    })
    eval: Dataset({
        features: ['Date', 'text', 'Sentiment'],
        num_rows: 9724
    })
})

In [7]:
train_ds = tweet_sent["train"]

In [8]:
train_ds

Dataset({
    features: ['Date', 'text', 'Sentiment'],
    num_rows: 77791
})

In [10]:
train_ds[0]

{'Date': '2019-06-12',
 'text': 'Are you interested in Bitcoin mining for FREE!?\nYou can start now if you already have a Bitcoin wallet.\nhttps://t.co/sCnP5klVBV\n#BITCOIN',
 'Sentiment': 'Positive'}

In [11]:
train_ds.column_names

['Date', 'text', 'Sentiment']

In [13]:
print(train_ds.features)

{'Date': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None), 'Sentiment': Value(dtype='string', id=None)}


In [14]:
print(train_ds['text'][:5])

['Are you interested in Bitcoin mining for FREE!?\nYou can start now if you already have a Bitcoin wallet.\nhttps://t.co/sCnP5klVBV\n#BITCOIN', 'QA Analyst - IT Services Company ( WESTBURY, United States )  - [ ➡ https://t.co/Ax3SKouPlo ] #Robotics #AI #ArtificailIntelligence #Robots #AI  #jobs #Hiring #Careers  #Cryptocurrency #Blockchain #BTC #BitCoin #ETH #crypto https://t.co/DY4vDGIyzp', '🌍22/05/2019🌍\nA Abakus a qual a NovaDAX pertence, foi selecionada como a marca com crescimento mais rápido da China na edição de 2019 do IAI Festival. #Blockchain #NovaDAX #Cripto #Stellar #Ripple #Tether #EOS #coinbr #minerarbitcoin #tron #BitTorrent #USDT  #Bitcoin #ETH https://t.co/RRA1ZY4mLS', '@Quark_Chain #QKC #Blockchain #BTC #ETH #blockchaintechnology #sharding\nыафвыв\nыв\nаы', 'July 21, 2018 04:00 AM EDT\nLast 4 hours, BTC 0.66% ETH 0.92% LTC 1.75% XRP 0.18% BCH 1.2% \n#cryptofinance #cryptocurrencies #BTC #ETH #LTC #XRP #BCH']


In [18]:
import pandas as pd
tweet_sent.set_format(type="pandas")
df = tweet_sent["train"][:]

In [20]:
df.head(10)

,Date,text,Sentiment
0,2019-06-12,Are you interested in Bitcoin mining for FREE!...,Positive
1,2019-07-08,"QA Analyst - IT Services Company ( WESTBURY, U...",Positive
2,2019-05-22,🌍22/05/2019🌍\nA Abakus a qual a NovaDAX perten...,Negative
3,2019-05-16,@Quark_Chain #QKC #Blockchain #BTC #ETH #block...,Negative
4,2018-07-21,"July 21, 2018 04:00 AM EDT\nLast 4 hours, BTC ...",Positive
5,2019-07-11,"As of July 10, 2019 at 09:50AM, 1 BTC = 11671....",Negative
6,2017-12-08,bitcoin priceってゆうか、,Negative
7,2019-05-14,FIRST 100 MEMBERS WILL BE ALLOWED\n\nJoin : ht...,Positive
8,2019-06-14,I want to drop a kind word in favour of to thi...,Positive
9,2019-07-01,https://t.co/utvdy6QR4r\nClick the link\nSign ...,Negative


In [21]:
#Remove stop words
#Remove @ and url

In [23]:
tweet_sent_df = df.drop(columns=['Date'])

In [24]:
tweet_sent_df.head(10)

,text,Sentiment
0,Are you interested in Bitcoin mining for FREE!...,Positive
1,"QA Analyst - IT Services Company ( WESTBURY, U...",Positive
2,🌍22/05/2019🌍\nA Abakus a qual a NovaDAX perten...,Negative
3,@Quark_Chain #QKC #Blockchain #BTC #ETH #block...,Negative
4,"July 21, 2018 04:00 AM EDT\nLast 4 hours, BTC ...",Positive
5,"As of July 10, 2019 at 09:50AM, 1 BTC = 11671....",Negative
6,bitcoin priceってゆうか、,Negative
7,FIRST 100 MEMBERS WILL BE ALLOWED\n\nJoin : ht...,Positive
8,I want to drop a kind word in favour of to thi...,Positive
9,https://t.co/utvdy6QR4r\nClick the link\nSign ...,Negative


In [25]:
tweet_sent_df["Sentiment"].unique()

array(['Positive', 'Negative', 'Neutral'], dtype=object)

In [28]:
!pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
                                              0.0/11.1 MB ? eta -:--:--
                                              0.0/11.1 MB ? eta -:--:--
                                             0.0/11.1 MB 435.7 kB/s eta 0:00:26
                                             0.0/11.1 MB 279.3 kB/s eta 0:00:40
                                             0.1/11.1 MB 290.5 kB/s eta 0:00:39
                                             0.1/11.1 MB 302.7 kB/s eta 0:00:37
                                             0.1/11.1 MB 348.6 kB/s eta 0:00:32
                                             0.1/11.1 MB 348.6 kB/s eta 0:00:32
                                             0.1/11.1 MB 348.6 kB/s eta 0:00:32
                                             0.1/11.1 MB 348.6 kB/s eta 0:00:32
                                             0.1/11.1 MB 348.6 kB/s eta 0:00:32
                                             0.1/11.1 MB 348.6 

In [29]:
from sklearn.preprocessing import LabelEncoder

In [35]:
#convert them to integers
def convert_str_int_col(df,column_name, custom_map=None):
    if custom_map:
        df[f"{column_name}_int"] = df[column_name].map(custom_map)
        mapping = custom_map
    else:
        Label_encoder = LabelEncoder()
        df[f"{column_name}_int"] = Label_encoder.fit_transform(df[column_name])
        mapping = dict(zip(Label_encoder.classes_,Label_encoder.transform(Label_encoder.classes_)))
    return df,mapping

In [37]:
custom_map = {"Positive":1,"Neutral":0, "Negative":2}
tweet_sent_df, mapping = convert_str_int_col(tweet_sent_df,"Sentiment",custom_map)

In [38]:
tweet_sent_df

,text,Sentiment,Sentiment_int
0,Are you interested in Bitcoin mining for FREE!...,Positive,1
1,"QA Analyst - IT Services Company ( WESTBURY, U...",Positive,1
2,🌍22/05/2019🌍\nA Abakus a qual a NovaDAX perten...,Negative,2
3,@Quark_Chain #QKC #Blockchain #BTC #ETH #block...,Negative,2
4,"July 21, 2018 04:00 AM EDT\nLast 4 hours, BTC ...",Positive,1
...,...,...,...
77786,RT workinbigdata: IT Systems Admin - Jupiter A...,Positive,1
77787,The strenght of your HODL determines the resis...,Positive,1
77788,The Future of the Web is Decentralized - #Info...,Negative,2
77789,Shut up and take my money! How dare you say th...,Positive,1


In [41]:
!pip install emoji nltk

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
                                              0.0/590.6 kB ? eta -:--:--
                                              10.2/590.6 kB ? eta -:--:--
     -                                     20.5/590.6 kB 165.2 kB/s eta 0:00:04
     -                                     20.5/590.6 kB 165.2 kB/s eta 0:00:04
     -                                     20.5/590.6 kB 165.2 kB/s eta 0:00:04
     -                                     30.7/590.6 kB 100.9 kB/s eta 0:00:06
     ---                                   51.2/590.6 kB 187.0 kB/s eta 0:00:03
     ---                                   61.4/590.6 kB 204.8 kB/s eta 0:00:03
     ----                                  71.7/590.6 kB 206.9 kB/s eta 0:00:03
     ------                               102.4/590.6 kB 245.8 kB/s eta 0:00:02
     ------                               102.4/590.6 kB 245.8 kB/s eta 0:00:02
     ---------                            153.6/590.6 kB 316

In [42]:
import re
import emoji
from nltk.corpus import stopwords
import torch.nn as nn

In [52]:
class FeatureCleaner(nn.Module):
    def __init__(self):
        super().__init__()
        #load stop words 
        self.stop_words = set(stopwords.words("english"))
        self.slang_dict = {
            "HODL": "hold on for dear life",
            "FOMO": "fear of missing out",
        }
    def remove_url(self,text):
        #Remove url
        return re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    def remove_hashtags(self,text):
        #Remove hastags and mentions
        text = re.sub(r"@\w+","",text)
        text = re.sub(r"#\w+","",text)
        return text
    def remove_special_characters(self,text):
        return re.sub(r"[^a-zA-Z0-9\s]","",text)
    def to_lowercase(self,text):
        return text.lower()
    def demoji(self,text):
        return emoji.demojize(text)
    def remove_stop_words(self,text):
        #remove stop words
        return " ".join([ word for word in text.split() if word not in self.stop_words])
    def expand_slangs(self,text):
        #expand slangs
        for word, full_form in self.slang_dict.items():
            text = text.replace(word,full_form)
        return text
    def remove_dates(self, text):
        """Removes dates in various formats from text."""
        # Matches common date formats like '2023-01-01', '01/20/2025', 'Jan 20, 2025', etc.
        return re.sub(
            r"\b\d{4}-\d{2}-\d{2}\b|\b\d{2}/\d{2}/\d{4}\b|\b\w{3,9}\s\d{1,2},?\s\d{4}\b", 
            "", 
            text
        )
    def forward(self,text,remove_stopwords=True):
        text = self.remove_url(text)
        text = self.remove_hashtags(text)
        text = self.remove_special_characters(text)
        text = self.to_lowercase(text)
        text = self.demoji(text)
        text = self.expand_slangs(text)
        text = self.remove_dates(text)
        if remove_stopwords:
            text = self.remove_stop_words(text)
        return text

In [53]:
feature_cleaner = FeatureCleaner()

In [54]:
tweet_sent_df['text'] = tweet_sent_df['text'].apply(lambda x: feature_cleaner(x))

In [55]:
tweet_sent_df.head(10)

,text,Sentiment,Sentiment_int
0,interested bitcoin mining free start already b...,Positive,1
1,qa analyst services company westbury united st...,Positive,1
2,22052019 abakus qual novadax pertence foi sele...,Negative,2
3,,Negative,2
4,0400 edt last 4 hours btc 066 eth 092 ltc 175 ...,Positive,1
5,0950am 1 btc 1167196 usd,Negative,2
6,bitcoin price,Negative,2
7,first 100 members allowed join cfi hmq crw vox...,Positive,1
8,want drop kind word favour project outstanding...,Positive,1
9,click link sign amp receive 25 instantly,Negative,2


In [64]:
tweet_sent_df.loc[3,"text"]

''

In [67]:
# Count the number of empty or NaN rows in the 'text' column
empty_row_count = tweet_sent_df['text'].isna().sum() + (tweet_sent_df['text'].str.strip() == "").sum()
print(f"Number of empty rows: {empty_row_count}")


Number of empty rows: 10792


In [68]:
# Drop rows where 'text', 'Sentiment', or 'Sentiment_int' are empty or NaN
tweet_sent_df = tweet_sent_df.dropna(subset=["text", "Sentiment", "Sentiment_int"])

# Remove rows where 'text' is an empty string after stripping whitespace
tweet_sent_df = tweet_sent_df[~(tweet_sent_df["text"].str.strip() == "")]


In [69]:
tweet_sent_df['text'].isna().sum()

np.int64(0)

In [70]:
tweet_sent_df.head(10)

,text,Sentiment,Sentiment_int
0,interested bitcoin mining free start already b...,Positive,1
1,qa analyst services company westbury united st...,Positive,1
2,22052019 abakus qual novadax pertence foi sele...,Negative,2
4,0400 edt last 4 hours btc 066 eth 092 ltc 175 ...,Positive,1
5,0950am 1 btc 1167196 usd,Negative,2
6,bitcoin price,Negative,2
7,first 100 members allowed join cfi hmq crw vox...,Positive,1
8,want drop kind word favour project outstanding...,Positive,1
9,click link sign amp receive 25 instantly,Negative,2
10,live profit 863 078 buy b295 37500 sell 37612,Positive,1
